In [1]:
import tensorflow as tf

2022-06-04 13:53:29.424016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-04 13:53:29.424080: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
modelo = tf.keras.models.load_model('rede1.h5')

2022-06-04 13:56:31.066701: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-04 13:56:31.079428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-06-04 13:56:31.160772: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-04 13:56:36.171956: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-04 13:56:36.533224: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-06-04 13:56:36

In [5]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________


In [128]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum=20220429"

In [129]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [135]:
df.head(40)

,campoX,ticker,diaNumPrevisao
0,"1.186948, 1.192386, 1.210741, 1.211421, 1.210...",ABEV3,20220429
1,"1.548828, 1.550781, 1.585937, 1.603516, 1.562...",AERI3,20220429
2,"1.939109, 1.955941, 1.989109, 2.025248, 1.996...",ALPA4,20220429
3,"0.936110, 0.932797, 0.952674, 0.973024, 0.966...",ALSO3,20220429
4,"1.841880, 1.867521, 1.944444, 1.970085, 1.961...",AMAR3,20220429
5,"1.211538, 1.202797, 1.230769, 1.253497, 1.253...",ANIM3,20220429
6,"1.199370, 1.207471, 1.240774, 1.262826, 1.242...",AZUL4,20220429
7,"0.881569, 0.864545, 0.865285, 0.869726, 0.876...",B3SA3,20220429
8,"0.856553, 0.853896, 0.861865, 0.870130, 0.869...",BBAS3,20220429
9,"1.128233, 1.113374, 1.127133, 1.127683, 1.129...",BBDC4,20220429


In [131]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [132]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(88, 1785)

In [133]:
dfTratadoX.head()

0    [1.186948, 1.192386, 1.210741, 1.211421, 1.210...
1    [1.548828, 1.550781, 1.585937, 1.603516, 1.562...
2    [1.939109, 1.955941, 1.989109, 2.025248, 1.996...
3    [0.93611, 0.932797, 0.952674, 0.973024, 0.9668...
4    [1.84188, 1.867521, 1.944444, 1.970085, 1.9615...
dtype: object

In [92]:
result = modelo.predict(X)

In [93]:
df['resultado'] = result
df['redeId'] = "1"

In [94]:
df.drop('campoX', inplace=True, axis=1)

In [95]:
json = df.to_dict(orient='list')

In [96]:
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisao'

response = requests.post(url, data=json)

In [97]:
print(type(data))

<class 'dict'>
